<a href="https://colab.research.google.com/github/Cellula-Tech/Machine-Learning-2024-Winter-Intern-/blob/main/Cellula2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [90]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer #Import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from imblearn.over_sampling import SMOTE
%matplotlib inline

In [91]:
df= pd.read_csv("/content/first inten project - first inten project.csv")
df.head()

,Booking_ID,number of adults,number of children,number of weekend nights,number of week nights,type of meal,car parking space,room type,lead time,market segment type,repeated,P-C,P-not-C,average price,special requests,date of reservation,booking status
0,INN00001,1,1,2,5,Meal Plan 1,0,Room_Type 1,224,Offline,0,0,0,88.00,0,10/2/2015,Not_Canceled
1,INN00002,1,0,1,3,Not Selected,0,Room_Type 1,5,Online,0,0,0,106.68,1,11/6/2018,Not_Canceled
2,INN00003,2,1,1,3,Meal Plan 1,0,Room_Type 1,1,Online,0,0,0,50.00,0,2/28/2018,Canceled
3,INN00004,1,0,0,2,Meal Plan 1,0,Room_Type 1,211,Online,0,0,0,100.00,1,5/20/2017,Canceled
4,INN00005,1,0,1,2,Not Selected,0,Room_Type 1,48,Online,0,0,0,77.00,0,4/11/2018,Canceled


In [92]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36285 entries, 0 to 36284
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Booking_ID                36285 non-null  object 
 1   number of adults          36285 non-null  int64  
 2   number of children        36285 non-null  int64  
 3   number of weekend nights  36285 non-null  int64  
 4   number of week nights     36285 non-null  int64  
 5   type of meal              36285 non-null  object 
 6   car parking space         36285 non-null  int64  
 7   room type                 36285 non-null  object 
 8   lead time                 36285 non-null  int64  
 9   market segment type       36285 non-null  object 
 10  repeated                  36285 non-null  int64  
 11  P-C                       36285 non-null  int64  
 12  P-not-C                   36285 non-null  int64  
 13  average price             36285 non-null  float64
 14  specia

In [93]:
df.nunique()

,0
Booking_ID,36285
number of adults,5
number of children,6
number of weekend nights,8
number of week nights,18
type of meal,4
car parking space,2
room type,7
lead time,352
market segment type,5


In [94]:
df.fillna(0, inplace=True)  # Replace missing values with 0

In [95]:
# Convert categorical variables to numerical
le = LabelEncoder()
df['booking status'] = le.fit_transform(df['booking status'])

In [96]:
# Feature scaling
scaler = StandardScaler()
df[['number of adults', 'number of children', 'average price']] = scaler.fit_transform(df[['number of adults', 'number of children', 'average price']])

In [97]:
# Convert 'date of reservation' to datetime and extract features
df['date of reservation'] = pd.to_datetime(df['date of reservation'], format='%m/%d/%Y', errors='coerce')
df['reservation_year'] = df['date of reservation'].dt.year
df['reservation_month'] = df['date of reservation'].dt.month
df['reservation_day'] = df['date of reservation'].dt.day
df = df.drop('date of reservation', axis=1)


In [98]:
# Split into features and target
X = df.drop(['booking status','Booking_ID'], axis=1)# Drop 'Booking_ID'
y = df['booking status']


In [99]:
# One-hot encode categorical features
X = pd.get_dummies(X, columns=['type of meal', 'room type', 'market segment type'])


In [100]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [101]:
#Impute missing values in X_train and X_test
imputer = SimpleImputer(strategy='mean') # or strategy='median', 'most_frequent'
X_train = imputer.fit_transform(X_train)
X_test = imputer.fit_transform(X_test)


In [102]:
# Create and train the model
model = LogisticRegression()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model (adjust metrics as needed)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8028110789582472


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [104]:
# Balancing the dataset using SMOTE (for oversampling the minority class)
smote = SMOTE(random_state=42)
X_train_sm, y_train_sm = smote.fit_resample(X_train, y_train)


In [105]:
# K-Nearest Neighbors Classifier with Hyperparameter Tuning
knn = KNeighborsClassifier()

# Parameter grid for KNN: trying different values of 'k' (number of neighbors)
knn_param_grid = {
    'n_neighbors': list(range(1, 31)),  # Test k values from 1 to 30
    'weights': ['uniform', 'distance'],  # Try uniform and distance-based weights
    'metric': ['euclidean', 'manhattan']  # Test different distance metrics
}


In [ ]:
# Hyperparameter Tuning with GridSearchCV for KNN
knn_grid_search = GridSearchCV(estimator=knn, param_grid=knn_param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=1)
knn_grid_search.fit(X_train_sm, y_train_sm)

# Best KNN model
best_knn_model = knn_grid_search.best_estimator_

# Make predictions with KNN
y_pred_knn = best_knn_model.predict(X_test)

Fitting 5 folds for each of 120 candidates, totalling 600 fits


In [ ]:
# Evaluate KNN model
knn_accuracy = accuracy_score(y_test, y_pred_knn)
print("KNN Accuracy after preprocessing and tuning:", knn_accuracy)
print("Best KNN Hyperparameters:", knn_grid_search.best_params_)